# Actividad Integradora

Diego Antonio García Padilla (A01710777)

## Leer expresión regular

Leer la expresión regular, parsearla y convertirla a una expresión postfija.

In [293]:
# Leer alfabeto

alfabeto_input = input("Introduce el alfabeto: ")

alfabeto = set()

for simbolo in alfabeto_input:
    alfabeto.add(simbolo)

In [294]:
# Leer expresión regular (se asume que es correcta)

regex_input = input("Ingrese la expresión regular: ").lower()

print("La expresión regular es: ", regex_input)

La expresión regular es:  (ab)+


In [295]:
# Parsear expresión regular
# Ejemplo: a*b(a|b) -> a*.b.(a|b)

# Formatear expresión regular
# O(n)

def format_regex(regex_input): 
    regex = ""
    
    # Agregar punto en concatenación
    for i in range(len(regex_input)):
        if i < len(regex_input) - 1:
            if regex_input[i] in alfabeto and regex_input[i + 1] in alfabeto:
                regex += regex_input[i] + "."
            elif regex_input[i + 1] == "(":
                regex += regex_input[i] + "."
            elif regex_input[i] == ")" and regex_input[i + 1] in alfabeto:
                regex += regex_input[i] + "."
            else:
                regex += regex_input[i]
        else:
            regex += regex_input[i]
        
    # Agregar punto entre operadores de repetición y sus operandos
    op_rep = ["*", "+"]
    
    
    for i in range(len(regex)):
        if i < len(regex) - 1:
            if regex[i] in op_rep and regex[i + 1] in alfabeto:
                regex = regex[:i + 1] + "." + regex[i + 1:]
            elif regex[i] in op_rep and regex[i + 1] == "(":
                regex = regex[:i + 1] + "." + regex[i + 1:]
    
    # Retornar expresión regular formateada
    return regex

# Formatear expresión regular dos veces para asegurar que no queden puntos juntos
regex = format_regex(regex_input)

        
print("RegEx: ", regex)

RegEx:  (a.b)+


In [296]:
# Parsear expresión regular a posfijo
# O(n)

def regex_a_postfix(regex):
    postfix = ""
    stack = []
    
    # Definir la prioridad de los operadores
    prioridad = {'*': 3, '+': 2, '.': 1}
    
    for char in regex:
        # Si el caracter es un símbolo del alfabeto, agregarlo al posfijo
        if char in alfabeto:
            postfix += char
            
        # Si el caracter es un paréntesis izquierdo, agregarlo al stack
        elif char == "(":
            stack.append(char)
        
        # Si el caracter es un operador, agregarlo al stack 
        elif char == ")":
            # Sacar operadores del stack hasta encontrar el paréntesis izquierdo
            while stack[-1] != "(":
                postfix += stack.pop()
            # Sacar el paréntesis izquierdo
            stack.pop()
        else:
            while stack and stack[-1] != "(" and prioridad[char] <= prioridad[stack[-1]]:
                postfix += stack.pop()
            stack.append(char)
    
    while stack:
        postfix += stack.pop()
    
    return postfix

postfix = regex_a_postfix(regex)

print("Posfijo: ", postfix)

Posfijo:  ab.+


## Generar NFA

Generar un autómata finito no determinista a partir de la expresión regular postfija.

In [297]:
# Generar autómata finito no determinista

expresiones = []
transiciones = {}

estados = -1
estado_inicial = 0
estado_final = 0

for char in postfix:
    # Si es un símbolo del alfabeto, generar un autómata finito no determinista con un estado inicial,
    # un estado final y una transición con el símbolo.
    if char in alfabeto:
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transición vacía de estado inicial a estado final
        transiciones[estado_inicial] = []
        
        # Agregar transición de estado inicial a estado final con el símbolo
        transiciones[estado_inicial].append([estado_final, char])
    
    # Si es un operador de cerradura de Kleene
    if char == "*":
        # Tomar la última expresión generada (para repetición es necesario tener una expresión)
        [e1, e2] = expresiones.pop()
        
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transiciones vacías
        transiciones[estado_inicial] = []
        if transiciones.get(e2) == None:
            transiciones[e2] = []
        
        # Agregar transiciones
        transiciones[estado_inicial].append([e1, "#"])
        transiciones[e2].append([estado_final, "#"])
        transiciones[estado_inicial].append([estado_final, "#"])
        transiciones[e2].append([e1, "#"])
        
    # Si es un operador de cerradura abierta
    if char == "+":
        # Tomar la última expresión generada (para repetición es necesario tener una expresión)
        [e1, e2] = expresiones.pop()
        
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transiciones vacías
        transiciones[estado_inicial] = []
        if transiciones.get(e2) == None:
            transiciones[e2] = []
        
        # Agregar transiciones
        transiciones[estado_inicial].append([e1, "#"])
        transiciones[e2].append([estado_final, "#"])
        transiciones[e2].append([e1, "#"])
    
    # Si es un operador de concatenación, unir los dos últimos autómatas finitos no deterministas generados
    if char == ".":
        # Tomar las dos últimas expresiones generadas (para concatenación es necesario tener dos expresiones)
        [e1, e2] = expresiones.pop() # 2do
        [e3, e4] = expresiones.pop() # 1er
        
        # Agregar transición vacia
        if transiciones.get(e4) == None:
            transiciones[e4] = []
        
        # Agregar transiciones
        transiciones[e4].append([e1, "#"])
        
        # Agregar estados a expresiones
        expresiones.append([e3, e2])
        
        # Actualizar estado inicial y final
        estado_inicial = e3
        estado_final = e2
    
    # Si es un operador de elección
    if char == "|":
        # Tomar las dos últimas expresiones generadas (para elección es necesario tener dos expresiones)
        [e1, e2] = expresiones.pop() # 2do
        [e3, e4] = expresiones.pop() # 1er
        
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transiciones vacías
        transiciones[estado_inicial] = []
        if transiciones.get(e2) == None:
            transiciones[e2] = []
        if transiciones.get(e4) == None:
            transiciones[e4] = []
        
        # Agregar transiciones
        transiciones[estado_inicial].append([e1, "#"])
        transiciones[estado_inicial].append([e3, "#"])
        transiciones[e2].append([estado_final, "#"])
        transiciones[e4].append([estado_final, "#"])
        
# Añadir estado final al diccionario de transiciones
# transiciones[estado_final] = []

# Ordenar transiciones
transiciones = dict(sorted(transiciones.items()))

# Imprimir resultado
print("Estado inicial: ", estado_inicial)
print("Estado final: ", estado_final)

for transicion in transiciones:
    print("Transición: ", transicion, " -> ", transiciones[transicion])

Estado inicial:  4
Estado final:  5
Transición:  0  ->  [[1, 'a']]
Transición:  1  ->  [[2, '#']]
Transición:  2  ->  [[3, 'b']]
Transición:  3  ->  [[5, '#'], [0, '#']]
Transición:  4  ->  [[0, '#']]
